In [2]:
import win32com.client
import numpy as np
import cv2
import time
cam = cv2.VideoCapture(0)
cam.set(3,1280)
cam.set(4,1024)
cam.set(15, 0.1)
speaker = win32com.client.Dispatch("SAPI.SpVoice")
classNames= []
classFile = 'C:/Users/hazemhassan/Desktop/Object-detection-using-opencv-master/Object-detection-using-opencv-master/coco.names'
with open(classFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
  

In [ ]:
modelConfiguration='C:/Users/hazemhassan/Desktop/Object-detection-using-opencv-master/Object-detection-using-opencv-master/yolov3-spp.cfg'
modelWeights = 'C:/Users/hazemhassan/Desktop/Object-detection-using-opencv-master/Object-detection-using-opencv-master/yolov3-spp.weights'
net = cv2.dnn.readNetFromDarknet (modelConfiguration, modelWeights)
net.setPreferableBackend (cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn. DNN_TARGET_CPU)
whT = 320
confThreshold = 0.5
nmsThreshold=0.3
def findobjects(outputs,img):
    hT, wT, cT = img.shape
    bbox=[]
    classIds = []
    confs = [] 
    for output in outputs:
        for det in output:
            scores =det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det [2]*wT),int(det[3]*hT) #21
                x,y = int((det[0]*wT)-w/2),int((det[1]*hT)-h/2)
                bbox.append ([x,y,w,h])
                classIds.append (classId)
                confs.append (float(confidence))
    indices = cv2.dnn. NMSBoxes (bbox, confs,confThreshold,nmsThreshold)
    for i in indices:
        box=bbox[i]
        x,y,w,h= box[0],box[1],box[2],box[3]
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),2) #31
        cv2.putText (img,f'{classNames[classIds[i]].upper ()} {int(confs[i]*100)}%',(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,255),2)
while True:
    success, img =cam.read()
    blob=cv2.dnn.blobFromImage(img,1/255, (whT,whT),[0,0,0],1,crop=False)
    net.setInput(blob)
    layerNames=net.getLayerNames()
    outputNames = [layerNames[i-1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputNames)
    findobjects(outputs,img)
    cv2.imshow('Image',img)
    cv2.waitKey(1)
    


In [ ]:
configPath = 'C:/Users/hazemhassan/Desktop/Object-detection-using-opencv-master/Object-detection-using-opencv-master/yolov3-spp.cfg'
weightPath = 'C:/Users/hazemhassan/Desktop/Object-detection-using-opencv-master/Object-detection-using-opencv-master/yolov3-spp.weights'
net =cv2.dnn_DetectionModel(weightPath ,configPath)
net.setInputSize(320,230)
net.setInputScale(1.0/127.5)
net.setInputMean((127.5, 127.5, 127.5))
speaks={}
cnt=0
while True:

    #cnt=cnt+1
    #print(cnt)
    success, img =cam.read()
    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (e,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs net.forward(output_layers_names)
    classIds, confs, bbox =net.detect(img, confThreshold = 0.5)
    if len(classIds)!= 0:
        for classId in classIds.flatten():
            print(type(classId-1))
            print(classId)
            if speaks.get(classId-1)==None:
                speaks[classId-1]=time.time()
                speaker.Speak(classNames[classId-1])
            else:
                if time.time()-speaks[classId-1]>=30:
                    speaks[classId-1]=None
                
            
    cv2.imshow('output',img)
    cv2.waitKey(1)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\layers\concat_layer.cpp:104: error: (-201:Incorrect size of input array) Inconsistent shape for ConcatLayer in function 'cv::dnn::ConcatLayerImpl::getMemoryShapes'
